https://www.youtube.com/watch?v=tcqEUSNCn8I
https://github.com/pixegami/langchain-rag-tutorial/blob/main/query_data.py

In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import os
import shutil
from langchain_community.llms import Ollama
import json
from langchain_core.prompts import ChatPromptTemplate

In [25]:
2311.12785
#paper_id = "2402.01383v1"
#paper_id = "2408.02304"
#paper_id = "2402.06196"
#paper_id = "2409.09957"
#paper_id = "2409.15180"
#paper_id = "2409.15816"
#paper_id = "2408.02464" #has problems
#paper_id = "2408.02085"
#paper_id = "2311.13731"
paper_id = "2311.12785"
CHROMA_PATH = paper_id+"/chroma"
#CHROMA_PATH = paper_id+"/chroma300"
DATA_PATH = paper_id
file = 'rag_largechroma'
#file = 'rag300'

In [26]:

def store_summary(survey1,filename):
    with open(f'summaries/'+paper_id+'/'+filename+'.txt', 'w') as file:
        file.write(survey1)
def write_json(documents_data,file_name):
    converted_data = [dict(item) for item in documents_data['source_documents']]
    dict_data = {'answer':documents_data['result'],'source':converted_data}
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)


### Create Chroma database

In [4]:

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

In [5]:
documents = load_documents()

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
documents[4]

Document(metadata={'source': '2311.12785/2305.18486v4.A_Systematic_Study_and_Comprehensive_Evaluation_of_ChatGPT_on_Benchmark_Datasets.pdf'}, page_content='3 2 0 2\n\nl u J\n\n5\n\n] L C . s c [\n\n4 v 6 8 4 8 1 . 5 0 3 2 : v i X r a\n\nA Systematic Study and Comprehensive Evaluation of ChatGPT on Benchmark Datasets\n\nMd Tahmid Rahman Laskar∗† §, M Saiful Bari∗‡, Mizanur Rahman∗† ¶ Md Amran Hossen Bhuiyan†, Shafiq Joty‡$, Jimmy Xiangji Huang† †York University, ‡Nanyang Technological University, §Dialpad Canada Inc., ¶Royal Bank of Canada, $Salesforce Research {tahmid20,mizanurr,amran,jhuang}@yorku.ca {bari0001,srjoty}@ntu.edu.sg\n\nAbstract\n\nThe development of large language models (LLMs) such as ChatGPT1 has brought a lot of attention recently. However, their evalua- tion in the benchmark academic datasets re- mains under-explored due to the difficulty of evaluating the generative outputs produced by this model against the ground truth. In this paper, we aim to present a thorough e

In [7]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,         #300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [8]:
chunks = split_text(documents)

Split 61 documents into 9718 chunks.
Least-to-most prompting can be combined with other prompting techniques like chain-of-thought (Wei et al., 2022) and self-consistency (Wang et al., 2022b), but does not need to be. Also, for some tasks, the two stages in least-to-most prompting can be merged to form a single-pass prompt.

3 RESULTS

We present least-to-most prompting results for symbolic manipulation, compositional generaliza- tion, and math reasoning tasks, and compare it with chain-of-thought prompting.

3.1 SYMBOLIC MANIPULATION
{'source': '2311.12785/2205.10625v3.Least_to_Most_Prompting_Enables_Complex_Reasoning_in_Large_Language_Models.pdf', 'start_index': 6884}


In [9]:
#
some = chunks[0:10]
#print(some)
print(len(chunks[11].page_content))
chunks[11].page_content

829


'3.1 SYMBOLIC MANIPULATION\n\nWe take the last-letter-concatenation task (Wei et al., 2022). In this task, each input is a list of words, and the corresponding output is the concatenation of the last letters of the words in the list. For example, “thinking, machine” outputs “ge”, since the last letter of “thinking” is “g” and the last letter of “machine” is “e”. Chain-of-thought prompting does a perfect job when the testing lists have the same length as the lists in the prompt exemplars. However, it performs poorly when the testing lists are much longer than the lists in the prompt exemplars. We show that least-to-most prompting overcomes this limitation and signiﬁcantly outperforms chain-of-thought prompting on length generalization.\n\nQ: “think, machine, learning” A: “think”, “think, machine”, “think, machine, learning”'

In [10]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_334729/2321294980.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
     #   shutil.rmtree(CHROMA_PATH)


    # Ensure the directory exists and has the correct permissions
    #os.makedirs(CHROMA_PATH, exist_ok=True)
    #os.chmod(CHROMA_PATH, 0o755)
        
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


In [12]:
save_to_chroma(chunks)

Saved 9718 chunks to 2311.12785/chroma.


/tmp/ipykernel_334729/2702161365.py:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


### Use database

In [27]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [28]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

### Other parameters
https://medium.com/@callumjmac/implementing-rag-in-langchain-with-chroma-a-step-by-step-guide-16fc21815339

In [15]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
Write your answer in about 2000 words.
"""

In [16]:
def query_rag(query_text,k=3):
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=k)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  llm = Ollama(model = "mistral",temperature=0)
  # Generate response text based on the prompt
  response_text = llm.invoke(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources} \nSourceText:{context_text}"
  return formatted_response, response_text



In [17]:
paper_id

'2311.12785'

In [29]:
def get_topic(file_name):
    # Step 1: Read the JSON file
    with open(file_name + '.json', 'r') as file:
        json_data = json.load(file)
    return json_data
original = get_topic('dataset/'+paper_id+'data')
topic = original['title']
print(topic)
db.similarity_search_with_relevance_scores(f'what are recent developments in {topic}?', k=2)

Prompting Frameworks for Large Language Models: A Survey


[(Document(metadata={'source': '2311.12785/2303.18223v15.A_Survey_of_Large_Language_Models.pdf', 'start_index': 698939}, page_content='[467] A. Prasad, P. Hase, X. Zhou, and M. Bansal, “Grips: Gradient-free, edit-based instruction search for prompting large language models,” in Proceedings of the 17th Conference of the European Chapter of the Association for Computational Linguistics, EACL 2023, Dubrovnik, Croatia, May 2-6, 2023, A. Vlachos and I. Augenstein, Eds. Association for Computational Linguistics, 2023, pp. 3827–3846.\n\n[468] Y. Zhou, A. I. Muresanu, Z. Han, K. Paster, S. Pitis, H. Chan, and J. Ba, “Large language models are human-level prompt engineers,” in The Eleventh International Conference on Learning Representations, ICLR 2023, Kigali, Rwanda, May 1-5, 2023. Open- Review.net, 2023. [469] R. Pryzant, D.\n\nIter,\n\nJ. Li, Y. T. Lee, C. Zhu,\n\nand M. Zeng, “Automatic prompt optimization with ”gradient descent” and beam search,” CoRR, vol. [Online]. Available: 2023. http

In [19]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=10)
# and finally, inspect our final response!
print(response_text)

Unable to find matching results.
 Recent developments in Prompting Frameworks for Large Language Models have gained significant attention due to their ability to effectively instruct and optimize the performance of these models without updating model parameters. In this context, we will discuss some of the recent advancements in prompt engineering and related research in the field of large language models.

Firstly, it is essential to understand the role of prompts in large language models. Prompts act as a natural language interface that stimulates the knowledge learned by Pre-trained Language Models (PLMs) during pre-training. They enable downstream tasks to be executed without updating model parameters for big models such as GPT-3. Moreover, with appropriate prompt guidance, models can perform complex reasoning tasks (Wei et al., 2022c; Wang et al., 2022a).

Prompt engineering plays a crucial role in the development of language models, providing a means to significantly augment thei

In [20]:
print(len(response_text.split(' ')))

747


In [21]:
os.makedirs('summaries/'+paper_id,exist_ok=True)
store_summary(formatted_response,'rag10_large_data') #'rag10'

In [22]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=5)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag5_large_data') #rag5

Unable to find matching results.
 Title: Recent Developments in Prompting Frameworks for Large Language Models: A Survey

Introduction

The advent of large language models (LLMs) has revolutionized the field of natural language processing (NLP), enabling machines to generate human-like text with remarkable accuracy and versatility. However, these models' performance heavily relies on the quality of the prompts provided during training or inference. Prompting is a crucial aspect of interacting with LLMs, as it determines the model's behavior and understanding of the task at hand. In recent years, researchers have explored various prompting frameworks to improve the effectiveness and efficiency of LLMs in handling diverse tasks. This survey aims to provide an overview of the latest developments in prompting frameworks for large language models.

Background: The Importance of Prompting in Large Language Models

Before delving into recent advancements, it is essential to understand the rol

In [23]:
print(len(response_text.split(' ')))

731


In [24]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=8)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag8_large_data') #rag8

Unable to find matching results.
 Recent developments in Prompting Frameworks for Large Language Models have gained significant attention in the field of Natural Language Processing (NLP) and Artificial Intelligence (AI), as these models have shown remarkable capabilities in various applications, from generating human-like text to performing complex reasoning tasks. In this context, we will discuss some recent surveys that provide insights into the current state-of-the-art in prompting frameworks for large language models.

Firstly, Qi et al. (2023) present a survey titled "A Survey for In-Context Learning," where they explore various aspects of in-context learning, which is closely related to prompting. They discuss the challenges and opportunities of in-context learning, as well as several methods and techniques that have been proposed to address these challenges. The authors also provide an overview of some popular datasets and benchmarks used for evaluating in-context learning mode

In [25]:
len(response_text.split(' '))

904

In [26]:
len('Hi what is the concept?'.split(' '))

5